# **Aplicación PLN para análisis y procesamiento de texto de carácter geopolítico, económico y político.**

### **1) COMPILACIÓN DEL CORPUS DE COMENTARIOS DE REDDIT A TRAVÉS DE SU API:**

**LIBRERÍAS**

In [4]:
import pandas as pd
import numpy as np
import requests
import json
import csv
import datetime as dt
import time

import praw
import pandas as pd
from datetime import datetime
import time
import re

**Primero de todo, extraeremos el corpus de documentos creando una aplicación para conectarnos a la API de Reddit y así poder extraer los datos.**

- Nombre de usuario: Solid_Valuable_2275
- Clave: 	iDAtL0KxYY9ky4GyPWyUM2OAZ4LnBg
- ID usuario: v4Ozy7so3ZR5n-i4r7Spqw

In [15]:
# Configuración de credenciales
reddit = praw.Reddit(
    client_id='v4Ozy7so3ZR5n-i4r7Spqw',
    client_secret='iDAtL0KxYY9ky4GyPWyUM2OAZ4LnBg',
    user_agent='dl-pln-2025-RGM-GLP',
    username='Solid_Valuable_2275',
    password='ABCdef12345%' 
)

In [16]:
import praw
import time


# Prueba simple de autenticación
try:
    # Intenta acceder a tu nombre de usuario para verificar autenticación
    print(f"Autenticado como: {reddit.user.me()}")
    
    # Prueba una solicitud simple
    for submission in reddit.subreddit("test").hot(limit=1):
        print(f"Título del post de prueba: {submission.title}")
        
    print("¡Autenticación exitosa!")
    
except Exception as e:
    print(f"Error de autenticación: {type(e).__name__} - {str(e)}")
    print("Por favor, verifica tus credenciales y la configuración de la aplicación.")

Error de autenticación: OAuthException - invalid_grant error processing request
Por favor, verifica tus credenciales y la configuración de la aplicación.


In [11]:
# Lista de subreddits a extraer
subreddits = ['Economics', 'investing', 'geopolitics', 'stocks', 'CryptoCurrency', 'business' ]



# Funciones de procesamiento de comentarios:

# Función para verificar si un texto es válido
def is_valid_comment(text):
    # Verificar longitud mínima (por ejemplo, 50 caracteres)
    if len(text) < 50:
        return False
    
    
    # Verificar si solo contiene URLs o emails
    url_pattern = re.compile(r'(https?://\S+)')
    email_pattern = re.compile(r'[\w\.-]+@[\w\.-]+')
    
    # Reemplazar URLs y emails con espacios
    text_without_urls = url_pattern.sub(' ', text)
    text_without_emails = email_pattern.sub(' ', text_without_urls)
    
    # Eliminar espacios múltiples y verificar contenido restante
    cleaned_text = ' '.join(text_without_emails.split())
    
    # Si queda poco texto después de eliminar URLs y emails, rechazar
    if len(cleaned_text) < 30:
        return False
    
    return True







# Para cada subreddit
for subreddit_name in subreddits:
    print(f"Procesando subreddit: {subreddit_name}.....")
    
    # Estructura para este subreddit
    subreddit_data = {
        "subreddit_name": subreddit_name,
        "threads": []
    }
    
    subreddit = reddit.subreddit(subreddit_name)
    
    # Obtener hilos de diferentes períodos temporales para asegurar diversidad
    time_filters = ['day', 'week', 'month', 'year', 'all']
    threads_collected = 0
    
    for time_filter in time_filters:
        if threads_collected >= 20:
            break
            
        # Calcular cuántos hilos necesitamos de este filtro
        threads_needed = min(4, 20 - threads_collected)
        
        if threads_needed <= 0:
            continue
            
        print(f"  Extrayendo de {subreddit_name} con filtro temporal: {time_filter}")
        
        try:
            threads = list(subreddit.top(time_filter=time_filter, limit=threads_needed))
            
            for thread in threads:
                if threads_collected >= 20:
                    break
                    
                print(f"    Procesando hilo: {thread.title[:30]}...")
                
                # Información del hilo
                thread_info = {
                    "thread_id": thread.id,
                    "title": thread.title,
                    "author": str(thread.author),
                    "created_utc": datetime.fromtimestamp(thread.created_utc).isoformat(),
                    "score": thread.score,
                    "url": thread.url,
                    "permalink": thread.permalink,
                    "time_filter": time_filter,
                    "comments": []
                }
                
                # Expandir todos los comentarios
                thread.comments.replace_more(limit=None)
                
                # Obtener comentarios planos
                flat_comments = thread.comments.list()
                
                # Contador para este hilo
                valid_comments_count = 0
                
                for comment in flat_comments:
                    # Verificar si el comentario es válido
                    if is_valid_comment(comment.body):

                        # Extraer información relevante
                        comment_info = {
                            'comment_id': comment.id,
                            'author': str(comment.author),
                            'text': comment.body,
                            'score': comment.score,
                            'created_utc': datetime.fromtimestamp(comment.created_utc).isoformat(),
                            'is_submitter': comment.is_submitter,
                            'permalink': comment.permalink
                        }
                        
                        thread_info["comments"].append(comment_info)
                        valid_comments_count += 1
                        
                        # Limitar a 50 comentarios válidos por hilo
                        if valid_comments_count >= 50:
                            break
                
                # Solo añadir hilos con suficientes comentarios válidos
                if valid_comments_count >= 25:  # Mínimo 25 comentarios válidos
                    subreddit_data["threads"].append(thread_info)
                    threads_collected += 1
                    print(f"      Añadido con {valid_comments_count} comentarios válidos")
                else:
                    print(f"      Descartado: solo {valid_comments_count} comentarios válidos")
                
                # Pausa para respetar límites de la API
                time.sleep(2)
        
        except Exception as e:
            print(f"Error en {subreddit_name} con filtro {time_filter}: {str(e)}")
            continue
    



    '''
    # Guardar datos de este subreddit en un archivo JSON
    output_file = os.path.join(output_dir, f"{subreddit_name}_data.json")
    
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(subreddit_data, f, ensure_ascii=False, indent=2)
    
    print(f"Completado: {subreddit_name} - {threads_collected} hilos guardados en {output_file}")

print("Extracción completa.")'''


Procesando subreddit: Economics.....
  Extrayendo de Economics con filtro temporal: day
Error en Economics con filtro day: invalid_grant error processing request
  Extrayendo de Economics con filtro temporal: week
Error en Economics con filtro week: invalid_grant error processing request
  Extrayendo de Economics con filtro temporal: month
Error en Economics con filtro month: invalid_grant error processing request
  Extrayendo de Economics con filtro temporal: year
Error en Economics con filtro year: invalid_grant error processing request
  Extrayendo de Economics con filtro temporal: all
Error en Economics con filtro all: invalid_grant error processing request
Procesando subreddit: investing.....
  Extrayendo de investing con filtro temporal: day
Error en investing con filtro day: invalid_grant error processing request
  Extrayendo de investing con filtro temporal: week
Error en investing con filtro week: invalid_grant error processing request
  Extrayendo de investing con filtro temp

In [9]:
print(subreddit_data)

{'subreddit_name': 'business', 'threads': []}


Verificación de comentarios válidos
Distribución temporal diversa
Descarte de hilos con pocos comentarios válidos



Pausas para respetar límites de la API
Manejo de excepciones para evitar interrupciones
Seguimiento del progreso mediante mensajes